# Imoprt CVS Information Page

Purpose of this Project: Extract Vaccine appointment info from the CVS. CVS data team has provide the links of appointment, slot and schedule. For more information please refer to https://developer.cvshealth.com/covid19/cdc/bulk-publish

Key Point:
Json File
Ndjson File


In [21]:
import requests
import json
import pandas as pd
import ndjson  ## Location Schedule and Slot are the Ndjson format
from datetime import datetime
from datetime import date

In [22]:
re=requests.get("https://www.cvs.com/immunizations/inventory/data/$bulk-publish")

In [23]:
# Check the status
re.status_code

200

In [24]:
re.json()

{'transactionTime': '2021-06-19T22:06:29.534Z',
 'request': 'https://www.cvs.com/immunizations/inventory/data/$bulk-publish',
 'output': [{'type': 'Location',
   'url': 'https://www.cvs.com/immunizations/inventory/data/location_file_06192021150629.ndjson'},
  {'type': 'Schedule',
   'url': 'https://www.cvs.com/immunizations/inventory/data/schedule_file_06192021150629.ndjson'},
  {'type': 'Slot',
   'url': 'https://www.cvs.com/immunizations/inventory/data/slot_file_06192021150629.ndjson'}],
 'error': []}

In [25]:
re.json()['output']

[{'type': 'Location',
  'url': 'https://www.cvs.com/immunizations/inventory/data/location_file_06192021150629.ndjson'},
 {'type': 'Schedule',
  'url': 'https://www.cvs.com/immunizations/inventory/data/schedule_file_06192021150629.ndjson'},
 {'type': 'Slot',
  'url': 'https://www.cvs.com/immunizations/inventory/data/slot_file_06192021150629.ndjson'}]

In [26]:
Type=[]
Url=[]
for i in re.json()['output']:
    Type.append(i['type'])
    Url.append(i['url'])

In [27]:
Type

['Location', 'Schedule', 'Slot']

In [28]:
Url

['https://www.cvs.com/immunizations/inventory/data/location_file_06192021150629.ndjson',
 'https://www.cvs.com/immunizations/inventory/data/schedule_file_06192021150629.ndjson',
 'https://www.cvs.com/immunizations/inventory/data/slot_file_06192021150629.ndjson']

In [29]:
df=pd.DataFrame(Url,columns=['Url'],index=Type)
#set Type as index

In [30]:
df.loc['Location']["Url"]  # becasue the type is Index

'https://www.cvs.com/immunizations/inventory/data/location_file_06192021150629.ndjson'

In [31]:
df.loc['Schedule']["Url"]

'https://www.cvs.com/immunizations/inventory/data/schedule_file_06192021150629.ndjson'

In [32]:
df.loc['Slot']['Url']

'https://www.cvs.com/immunizations/inventory/data/slot_file_06192021150629.ndjson'

In [33]:
requests.get(df.loc['Location']["Url"])

<Response [200]>

# Import

In [34]:
Location_re=requests.get(df.loc['Location']["Url"])
# there are ndjson file
#Location_re.json()     will return no value, so I used .text without .json()
Schedule_re=requests.get(df.loc['Schedule']["Url"])
Slot_re=requests.get(df.loc['Slot']["Url"])

In [35]:
#Location_re.text

## Convert String to Ndjson Format

### Location

In [ ]:
ndjson.loads(Location_re.text)[0]
#The ndjson file are different from json file, the boundary is ",", but also be careful of the ","

In [ ]:
ndjson.loads(Location_re.text)[0].keys()

In [ ]:
ndjson.loads(Location_re.text)[0]['telecom']
#type(ndjson.loads(Location_re.text)[0]['telecom'])  ## It is the list

In [ ]:
ndjson.loads(Location_re.text)[0]['telecom'][0]
#type(ndjson.loads(Location_re.text)[0]['telecom'][0]) ## It is the dict

In [ ]:
ndjson.loads(Location_re.text)[0]['telecom'][0]['value']

In [ ]:
ndjson.loads(Location_re.text)[0]['address']['line'][0]

### Schedule

In [ ]:
ndjson.loads(Schedule_re.text)[0]

In [ ]:
ndjson.loads(Schedule_re.text)[0].keys()

In [ ]:
ndjson.loads(Schedule_re.text)[0]['actor'][0]['reference'].split('/')[1]
# In the reference: the location/9714, we only need the number after /

In [ ]:
ndjson.loads(Schedule_re.text)[0]['id']

### Slot

In [ ]:
ndjson.loads(Slot_re.text)[0]

In [ ]:
ndjson.loads(Slot_re.text)[0].keys()

In [ ]:
ndjson.loads(Slot_re.text)[0]['schedule']['reference'].split('/')[1]

In [ ]:
ndjson.loads(Slot_re.text)[0]['status']

In [ ]:
ndjson.loads(Slot_re.text)[0]['start']

In [ ]:
ndjson.loads(Slot_re.text)[0]['end']

In [ ]:
ndjson.loads(Slot_re.text)[0]['extension'][1]['valueInteger']

## Create For Loop to organize information of Locations

In [36]:
Locations_Info=[]
for Locations in ndjson.loads(Location_re.text):
    Detail_Info={'id':Locations['id'],
     'Line':Locations['address']['line'][0],
     'City':Locations['address']['city'],
     'State':Locations['address']['state'],
     'PostalCode':Locations['address']['postalCode'],
         }
    Locations_Info.append(Detail_Info)

In [37]:
#Locations_Info

In [38]:
Location_DF=pd.DataFrame(Locations_Info)  ## Becasue Locations_Info is the List and dic is inside of this List

In [39]:
Location_DF

,id,Line,City,State,PostalCode
0,2502,581 HIGHLAND AVE.,CHESHIRE,CT,06410
1,1521,9140 ROTHBURY DRIVE,GAITHERSBURG,MD,20886
2,4365,709 W. COLLEGE ST.,PULASKI,TN,38478
3,1087,20 FILORS LN.,STONY POINT,NY,10980
4,4984,863 BALTIMORE PIKE,GLEN MILLS,PA,19342
...,...,...,...,...,...
9715,3634,2160 LEE RD,CLEVELAND HEIGHTS,OH,44118
9716,2535,49 CLAREMONT AVE.,MONTCLAIR,NJ,07042
9717,8875,3644 EAST OLYMPIC BOULEVARD,LOS ANGELES,CA,90023
9718,675,159 N. MACDADE BLVD.,GLENOLDEN,PA,19036


In [ ]:
Location_DF['State'].value_counts()
#Location_DF['State'].value_counts().sort_values()

In [ ]:
Location_DF.groupby(["State","City"]).count().reset_index()

## Create For Loop to organize information of Schedules

In [ ]:
Schedule_Info=[]
for Schedules in ndjson.loads(Schedule_re.text):
    Detail_Info={'id':Schedules['id'],
                 'actor':Schedules['actor'][0]['reference'].split('/')[1]
         }
    Schedule_Info.append(Detail_Info)

In [ ]:
 Schedule_Info

In [ ]:
Schedule_DF=pd.DataFrame(Schedule_Info)

In [ ]:
Schedule_DF['id'].value_counts()

In [ ]:
Schedule_DF['actor'].value_counts()
#Schedule_DF['actor'].value_counts().max()    # 1

## Create For Loop to organize information of Slots

In [ ]:
Slot_Info=[]
for Slots in ndjson.loads(Slot_re.text):
    Detail_Info={'id':Slots['schedule']['reference'].split('/')[1],
                 'Status':Slots['status'],
                 'Start':Slots['start'],
                 'End':Slots['end'],
                 'ValueInteger':Slots['extension'][1]['valueInteger']
         }
    Slot_Info.append(Detail_Info)

In [ ]:
 Slot_Info

In [ ]:
Slot_DF=pd.DataFrame(Slot_Info)

In [ ]:
Slot_DF

In [ ]:
Slot_DF['id'].value_counts()
#Slot_DF['id'].value_counts().max()    ## 1
#The id is the unique value

In [ ]:
Slot_DF['Start']=[datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ') for x in Slot_DF['Start']]

In [ ]:
Slot_DF['End']=[datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ') for x in Slot_DF['End']]

In [ ]:
Slot_DF[['Status','ValueInteger']].drop_duplicates()
#Slot_DF['Status'].value_counts()
#Slot_DF['ValueInteger'].value_counts()
#Based on the result, the busy means 0, free means 1

In [ ]:
Slot_DF[['Start','End']].drop_duplicates()

# Combine 3 DataFrames Together

The location files are the all locations of CVS
The Schedule tels service type and locations(all of them the service types are same)
The slot file tells the free or busy 

In [ ]:
Location_DF

In [ ]:
Schedule_DF

In [ ]:
Slot_DF

In [ ]:
Schedule_DF['id'].isin(Slot_DF['id']).value_counts()
#True    9564
#Name: id, dtype: int64

In [ ]:
Location_DF.merge(Schedule_DF,how='left',left_on='id',right_on='actor')

In [ ]:
Schedule_DF.merge(Slot_DF,how='inner',left_on='id',right_on='id')

In [ ]:
DF1=Location_DF.merge(Schedule_DF,how='left',left_on='id',right_on='actor')
DF=DF1.merge(Slot_DF,how='inner',left_on='id_y',right_on='id')

In [ ]:
DF=DF.drop(columns=['id_x','id_y'])

In [ ]:
DF.to_csv('CVS_File.csv',index=False)

# Analysis

In [ ]:
DF

In [ ]:
pd.pivot_table(DF,values='id',index='State',columns='Status',aggfunc='count')

# Refresh Data Every 15 minutes 

In [ ]:
import time

In [ ]:
# This part is to check if the number of locations changes. So I set up 15 minutes break for each loop. 
#Which took about 10 hours total. All of the output are 9737 so in total, there is only 9737 locations.

In [ ]:
i=1
while i<50:
    Location_re=requests.get(df.loc['Location']["Url"])
    Schedule_re=requests.get(df.loc['Schedule']["Url"])
    Slot_re=requests.get(df.loc['Slot']["Url"])
    
    Locations_Info=[]
    for Locations in ndjson.loads(Location_re.text):
        Detail_Info={'id':Locations['id'],
                     'Line':Locations['address']['line'][0],
                     'City':Locations['address']['city'],
                     'State':Locations['address']['state'],
                     'PostalCode':Locations['address']['postalCode'],
                    }
        Locations_Info.append(Detail_Info)
    
    Schedule_Info=[]
    for Schedules in ndjson.loads(Schedule_re.text):
        Detail_Info={'id':Schedules['id'],
                     'actor':Schedules['actor'][0]['reference'].split('/')[1]
                    }
        Schedule_Info.append(Detail_Info)    
    
    Slot_Info=[]
    for Slots in ndjson.loads(Slot_re.text):
        Detail_Info={'id':Slots['schedule']['reference'].split('/')[1],
                     'Status':Slots['status'],
                     'Start':Slots['start'],
                     'End':Slots['end'],
                     'ValueInteger':Slots['extension'][1]['valueInteger']
                    }
        Slot_Info.append(Detail_Info)
    
    
    Location_DF=pd.DataFrame(Locations_Info)
    Schedule_DF=pd.DataFrame(Schedule_Info)
    Slot_DF=pd.DataFrame(Slot_Info)
    DF1=Location_DF.merge(Schedule_DF,how='left',left_on='id',right_on='actor')
    DF=DF1.merge(Slot_DF,how='inner',left_on='id_y',right_on='id')
    DF=DF.drop(columns=['id_x','id_y'])
    DF.to_csv("{}_file_at_time_{}.csv".format(i,datetime.now().time().strftime("%H_%M_%S")),index=False)

    
    print(i)
    print("There are {} different locations".format(Location_DF.shape[0]))
    time.sleep(900)
    i+=1


In [ ]:
datetime.now()

In [ ]:
"DF_at {}".format(datetime.now().time().strftime("%H:%M"))

In [ ]:
print("There are {} different locations".format(Location_DF.shape[0]))

# Connect to PostgreSQL

In [156]:
import psycopg2

In [157]:
connection=psycopg2.connect(user="postgres",password="152068",host="localhost",port="5440",dbname="CVS Data ")

In [158]:
cur=connection.cursor()

In [ ]:
#connection.close

## Create Tables

In [119]:
# It is an example
create_table_query = '''CREATE TABLE Location
          (Id             INT PRIMARY KEY     NOT NULL,
           Line           TEXT    NOT NULL,
           City           TEXT,
           State          TEXT,
           PostalCode     INT); '''

In [120]:
cur.execute(create_table_query)

## Insert Data To Table

In [121]:
insert_query='''INSERT INTO location VALUES (%s,%s,%s,%s,%s)'''

In [122]:
for i in Location_DF.iloc:
    cur.execute(insert_query,i)

In [125]:
connection.commit()

## Extract Data

In [173]:
#query="select * from Location"
query="select state, count(*) from location group by state"

In [174]:
cur.execute(query)

In [175]:
#connection.commit()

In [176]:
#Print the first row only
#cur.fetchone()

In [177]:
#Print all rows if we do not run fetchone
cur.fetchall()

[('CA', 1155),
 ('NH', 47),
 ('OR', 22),
 ('ND', 6),
 ('TX', 835),
 ('PR', 22),
 ('NV', 91),
 ('KY', 81),
 ('OH', 385),
 ('NY', 556),
 ('HI', 69),
 ('NM', 24),
 ('MS', 57),
 ('IN', 327),
 ('DC', 59),
 ('NE', 30),
 ('WV', 57),
 ('MO', 197),
 ('FL', 834),
 ('ME', 25),
 ('AR', 23),
 ('WI', 80),
 ('NC', 361),
 ('CT', 178),
 ('SD', 3),
 ('OK', 75),
 ('RI', 61),
 ('ID', 3),
 ('GA', 339),
 ('MN', 128),
 ('PA', 475),
 ('AK', 3),
 ('MD', 224),
 ('WY', 2),
 ('LA', 134),
 ('MT', 14),
 ('IL', 372),
 ('TN', 158),
 ('WA', 45),
 ('MI', 296),
 ('NJ', 342),
 ('MA', 400),
 ('AL', 179),
 ('UT', 29),
 ('IA', 38),
 ('VT', 9),
 ('CO', 51),
 ('SC', 206),
 ('VA', 346),
 ('DE', 20),
 ('AZ', 194),
 ('KS', 53)]